# Moana thredds server connector

Author: Sebastien Delaux

This notebook describes how to get observation data from the thredds server set on top of the Moana 25 year hydrodynamic hindcast of New Zealand waters. The data is freely available from that server under the terms of the Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License (see https://www.moanaproject.org/data for more).

First we install xarray which we will use to handle the data

In [5]:
import sys
!{sys.executable} -m pip install xarray

  Using cached https://files.pythonhosted.org/packages/e3/25/cc8ccc40d21638ae8514ce2aef1f1db3036e31c2adea797c7501302726fa/xarray-0.15.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/62/20/4d43e141b5bc426ba38274933ef8e76e85c7adea2c321ecf9ebf7421cedf/numpy-1.18.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/08/ec/b5dd8cfb078380fb5ae9325771146bccd4e8cad2d3e4c72c7433010684eb/pandas-1.0.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e7/f9/f0b53f88060247251bf481fa6ea62cd0d25bf1b11a87888e53ce5b7c8ad2/pytz-2019.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/65/eb/1f97cb97bfc2390a276969c6fae16075da282f5058082d4cb10c6c5c1dba/six-1.14.0-py2.py3-none-any.whl


Now we load xarray

In [8]:
import xarray as xr

Then we install siphon which we will use to talk with the thredds server catalog

In [ ]:
!{sys.executable} -m pip install siphon

Now we load siphon

In [4]:
from siphon.catalog import TDSCatalog

Then we load a couple of reasonably standard python library

In [5]:
from datetime import datetime
import itertools

We define the connector

In [9]:
class MoanaThreddsServerConnector(object):
    """
    A class used to connect one the thredds server put
    on top of the Moana hydrodynamic hindcast of New Zealand waters
    
    Attributes
    ----------
    catalog_url : str
        The url to use to interogate the data catalog
    dods_base_url : str
        The base url to use to access the data via OpenDAP
        
    Methods
    -------
    get_point_data()
        return data at a given location
    get_grid_data()
        return data over a given bounding box
    """
    def __init__(self,
                 catalog_url='http://thredds.moanaproject.org:8080/thredds/catalog/moana/ocean/NZB/v1.9/avg/catalog.xml',
                 dods_base_url='http://thredds.moanaproject.org:8080/thredds/dodsC/moana/ocean/NZB/v1.9/avg/'):
        
        
        # Store the urls
        self.catalog_url = catalog_url
        self.dods_base_url = dods_base_url
        
        # Get catalog
        self.catalog = TDSCatalog(catalog_url)
        
        # Build list of all available files
        self.dods_url = [dods_base_url+filename\
                         for filename in sorted(self.catalog.datasets)]
        
        # Load base dataset object
        self.dset = xr.open_mfdataset(self.dods_url,
                                      combine='nested', concat_dim='ocean_time',
                                      data_vars='minimal', coords='minimal', compat='override',
                                      chunks={'eta_rho':16, 'xi_rho':16, 's_rho':1, 'ocean_time':16})
            
    def get_point_data(self, longitude, latitude, variables, tmin=None, tmax=None):
        
        to_drop = []
        if  variables:
            variables = variables + list(set(list(itertools.chain.from_iterable([list(dset.variables[var].dims) for var in variables]))))
            to_drop = [v for v in self.dset.variables.keys() if v not in variables]
        
        return self.dset.drop_vars(to_drop).sel(ocean_time=slice(tmin,tmax))

Initialise the connector

In [10]:
cat_url = "http://thredds.moanaproject.org:8080/thredds/catalog/moana/ocean/NZB/v1.9/avg/catalog.xml"
base_dods_url = 'http://thredds.moanaproject.org:8080/thredds/dodsC/moana/ocean/NZB/v1.9/avg/'

connector = MoanaThreddsServerConnector(cat_url, base_dods_url)

Get some data at a specific location

In [11]:
dset = connector.get_point_data(longitude=0., latitude=0,
                                variables=['u_eastward', 'v_northward'],
                                tmin=datetime(2017,1,1),
                                tmax=datetime(2017,1,2))
print(connector.dset.ocean_time.data[0], connector.dset.ocean_time.data[-1])
print(dset)

NameError: name 'dset' is not defined